# Capstone Final Project -- Morristown

## Introduction 

####  The town of Morristown is located in Morris county New Jersey. Eventhough, it is not considered a city like Hoboken, Jersey city or Newark. I guess one can considered it a town with a city feel with the number of many diverse venues. There are also a couple of universities nearby like FDU and Drew University. A couple of Forbes 500 companies have establish offices around the area.

## Business Problem

####  For this project, I want provide an overview of Morristowns venues like resturants and rank them accordingly. I will accomplish this by retieving the data using Foursquare API. I will be able to retrieve the Venue's Name, ID, Location, Category, count of likes and so on. I will take this data to seperate the venues into seperate clusters to rank the venues. I will also the geolocator to aquire the coordinates of Morristown to make the task easier.


## Preperation of Data


#### This is when necessary packages are imported and retrieve the data.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors


# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#import beautiful soup
from urllib.request import urlopen
from bs4 import BeautifulSoup


print('Libraries imported.')


Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


### Finding the geograhic data for Morristown.
#### The reason for this is to retrieve data from FourSquare for this town.

In [2]:
address = 'Morristown, New Jersey'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Morristown are {}, {}.'.format(latitude, longitude))


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Morristown are 40.7970384, -74.4809492.



### Part 1 -- FourSquare 
#### The Foursquare API to access it.


In [3]:
CLIENT_ID = 'RBXPCZTNAWJIEFEFACV1C5BGLLS2DLA5XETAL4A2AHFSJD0R' # your Foursquare ID
CLIENT_SECRET = 'P31JNEWFM40EQTE25TJNZMQADAHCAUVLG4JPBNG2MALJ1Y0I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version



print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: RBXPCZTNAWJIEFEFACV1C5BGLLS2DLA5XETAL4A2AHFSJD0R
CLIENT_SECRET:P31JNEWFM40EQTE25TJNZMQADAHCAUVLG4JPBNG2MALJ1Y0I


### Part 2
#### The URL for the venues in Morristown.

In [4]:

LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?&client_id=RBXPCZTNAWJIEFEFACV1C5BGLLS2DLA5XETAL4A2AHFSJD0R&client_secret=P31JNEWFM40EQTE25TJNZMQADAHCAUVLG4JPBNG2MALJ1Y0I&v=20180605&ll=40.7970384,-74.4809492&radius=500&limit=100'

### Part 3
#### Retrieving the JSON for the URL.

In [5]:
results = requests.get(url).json()
results


{'meta': {'code': 200, 'requestId': '5d9ed64aacc5f5002cd53c43'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Morristown',
  'headerFullLocation': 'Morristown',
  'headerLocationGranularity': 'city',
  'totalResults': 61,
  'suggestedBounds': {'ne': {'lat': 40.8015384045, 'lng': -74.47501600128913},
   'sw': {'lat': 40.792538395499996, 'lng': -74.48688239871086}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54bc2c76498e852a0330576d',
       'name': 'The Committed Pig',
       'location': {'address': '28 W Park Pl',
        'crossStreet': 'Route 202',
        'lat': 40.796839145558565,
        'lng': -74.48177613710573,
        'labeledLatLngs': [{'label': 'display',
          'l

### Part 4
#### Developing the Dataframe.

In [6]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
 

In [7]:
#pull the actual data from the Foursquare API

venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues


,venue.name,venue.id,venue.categories,venue.location.lat,venue.location.lng
0,The Committed Pig,54bc2c76498e852a0330576d,Burger Joint,40.796839,-74.481776
1,i do... i do...,4d55c72fcc65a14321d0525e,Bridal Shop,40.795381,-74.480001
2,HOPS,4d3b8b99a2e4b1f7a7f0e025,Bar,40.796812,-74.482786
3,The Morristown Green,4b8bf04ff964a5207bb332e3,Park,40.796398,-74.481125
4,Morristown Game Vault,56e9acbf498e2468f79a0097,Arcade,40.795845,-74.480250
5,Laundromat,56f5dfd9498e0f359934092b,Speakeasy,40.795506,-74.480535
6,Sushi Lounge,4b32ab3af964a520661125e3,Sushi Restaurant,40.797207,-74.482969
7,Morristown Pancake House,5442a6ea498e4795b0e18925,American Restaurant,40.795865,-74.480514
8,The Artist Baker,4b71d8c7f964a520cc5f2de3,Café,40.799101,-74.481801
9,Millies Old World Meatballs And Pizza,50e8922ee4b097a7b77a7224,Pizza Place,40.794923,-74.479071


In [8]:
#fix the column names

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues


,name,id,categories,lat,lng
0,The Committed Pig,54bc2c76498e852a0330576d,Burger Joint,40.796839,-74.481776
1,i do... i do...,4d55c72fcc65a14321d0525e,Bridal Shop,40.795381,-74.480001
2,HOPS,4d3b8b99a2e4b1f7a7f0e025,Bar,40.796812,-74.482786
3,The Morristown Green,4b8bf04ff964a5207bb332e3,Park,40.796398,-74.481125
4,Morristown Game Vault,56e9acbf498e2468f79a0097,Arcade,40.795845,-74.480250
5,Laundromat,56f5dfd9498e0f359934092b,Speakeasy,40.795506,-74.480535
6,Sushi Lounge,4b32ab3af964a520661125e3,Sushi Restaurant,40.797207,-74.482969
7,Morristown Pancake House,5442a6ea498e4795b0e18925,American Restaurant,40.795865,-74.480514
8,The Artist Baker,4b71d8c7f964a520cc5f2de3,Café,40.799101,-74.481801
9,Millies Old World Meatballs And Pizza,50e8922ee4b097a7b77a7224,Pizza Place,40.794923,-74.479071


In [9]:
# find a list of unique categories from the API so we can see what may or may not fit for restaurants

nearby_venues['categories'].unique()


array(['Burger Joint', 'Bridal Shop', 'Bar', 'Park', 'Arcade',
       'Speakeasy', 'Sushi Restaurant', 'American Restaurant', 'Café',
       'Pizza Place', 'Farmers Market', 'Dessert Shop',
       'Salon / Barbershop', 'Beer Bar', 'Athletics & Sports',
       'Indian Restaurant', 'Yoga Studio', 'Department Store',
       'Gym / Fitness Center', 'Steakhouse', 'Frozen Yogurt Shop',
       'Performing Arts Venue', 'Movie Theater', 'Mexican Restaurant',
       'Cosmetics Shop', 'Empanada Restaurant', 'Market', 'Coffee Shop',
       'Thai Restaurant', 'Ice Cream Shop', 'Mediterranean Restaurant',
       'Sporting Goods Shop', 'New American Restaurant',
       'Persian Restaurant', 'Bagel Shop', 'Juice Bar',
       'Furniture / Home Store', 'Fried Chicken Joint',
       'Italian Restaurant', 'Japanese Restaurant',
       'Modern European Restaurant'], dtype=object)

In [10]:
# creating a list of categories to remove from our dataframe because they are not restaurants. Did this manually.


removal_list = ['Gym / Fitness Center', 'Furniture / Home Store','Bridal Shop', 'Movie Theater', 'Market', 'Bakery', 'Park', "Women's Store", 'Sporting Goods Shop', 'Dog Run', 'Gaming Cafe',
               'Optical Shop', 'Farmers Market', 'Yoga Studio', 'Market', 'Pet Store', 'Shoe Repair','Trail','Food','Athletics & Sports', 'Jewelry Store', 'Record Shop', 'Juice Bar', 
               'Cosmetics Shop', 'Business Service', 'Salon / Barbershop', 'Liquor Store', 'Grocery Store', 'Stationery Store',
               'Pilates Studio','Department Store', 'Performing Arts Venue', 'Dessert Shop', 'Bookstore','Arcade', 'Concert Hall', 'Video Game Store', 'Pharmacy', 'Mobile Phone Shop',
               'Deli / Bodega']

nearby_venues2 = nearby_venues.copy()


#clear dataframe of just restaurants
nearby_venues2 = nearby_venues2[~nearby_venues2['categories'].isin(removal_list)]
nearby_venues2


,name,id,categories,lat,lng
0,The Committed Pig,54bc2c76498e852a0330576d,Burger Joint,40.796839,-74.481776
2,HOPS,4d3b8b99a2e4b1f7a7f0e025,Bar,40.796812,-74.482786
5,Laundromat,56f5dfd9498e0f359934092b,Speakeasy,40.795506,-74.480535
6,Sushi Lounge,4b32ab3af964a520661125e3,Sushi Restaurant,40.797207,-74.482969
7,Morristown Pancake House,5442a6ea498e4795b0e18925,American Restaurant,40.795865,-74.480514
8,The Artist Baker,4b71d8c7f964a520cc5f2de3,Café,40.799101,-74.481801
9,Millies Old World Meatballs And Pizza,50e8922ee4b097a7b77a7224,Pizza Place,40.794923,-74.479071
11,Asahi Sushi,4b7c7b55f964a5201a962fe3,Sushi Restaurant,40.797462,-74.477700
13,Masago,516f0e7ce4b0ee5002c9ed70,Sushi Restaurant,40.797772,-74.483184
15,"Revolution, A Social Brew House",57fbfd19498e5fcfe65ba2b3,Beer Bar,40.796002,-74.480858


### Part 5
#### Retrieve the list of venue of ids.

In [11]:
#list of venues

venue_id_list = nearby_venues2['id'].tolist()
venue_id_list


['54bc2c76498e852a0330576d',
 '4d3b8b99a2e4b1f7a7f0e025',
 '56f5dfd9498e0f359934092b',
 '4b32ab3af964a520661125e3',
 '5442a6ea498e4795b0e18925',
 '4b71d8c7f964a520cc5f2de3',
 '50e8922ee4b097a7b77a7224',
 '4b7c7b55f964a5201a962fe3',
 '516f0e7ce4b0ee5002c9ed70',
 '57fbfd19498e5fcfe65ba2b3',
 '4b69fdb8f964a52034c02be3',
 '4b92a71df964a520f70d34e3',
 '4ade3b40f964a5201e7421e3',
 '5570eb60498e84218b6ae325',
 '4d51fd29e0275481c069b0b6',
 '4ed7fa938b81b2bf292fc3c3',
 '4b4e1922f964a52056e126e3',
 '4aaad8bcf964a520b15720e3',
 '4e1b9793c65be9fffd74c4cb',
 '4b7ad359f964a520123f2fe3',
 '5928bce5d41bb74f6ec42252',
 '4a99f0fdf964a520223120e3',
 '563d26d1cd106e7049a26c7d',
 '4ad8ae12f964a520aa1321e3',
 '5328b1c911d2e167c91df448',
 '4dcb1c7345dd3831227dfe2d',
 '4b7567b6f964a520a30a2ee3',
 '51a0bb2a498e5f40f8a924de',
 '4a99a976f964a520ae2f20e3',
 '4b57ea3df964a520184628e3',
 '52cb5dbd498e7d6760b5b6b8',
 '4bdcac912a3a0f477ce6b3b6',
 '4fb05f9fe4b0fd09a9d72dd5',
 '4bae2ad7f964a520b58c3be3',
 '55da57b8498e

In [12]:

# pulling the likes from the API based on venue ID

url_list = []
like_list = []
json_list = []

for i in venue_id_list:
    venue_url = 'https://api.foursquare.com/v2/venues/{}/likes?client_id={}&client_secret={}&v={}'.format(i, CLIENT_ID, CLIENT_SECRET, VERSION)
    url_list.append(venue_url)
for link in url_list:
    result = requests.get(link).json()
    likes = result['response']['likes']['count']
    like_list.append(likes)
print(like_list)


[133, 54, 39, 58, 39, 19, 85, 47, 24, 37, 18, 30, 103, 71, 54, 18, 43, 42, 109, 21, 7, 16, 7, 70, 101, 31, 37, 24, 18, 44, 26, 14, 24, 9, 7, 18, 6, 4, 0, 1]


In [13]:

#double check that we did not lose any venues based on if likes were available

print(len(like_list))
print(len(venue_id_list))


40
40


### Data Prep --  Intro
#### This is where the data is retieve and seperate into different categories. I will then bin this data into a quality categorical variables so we can cluster appropriately.


In [14]:
#let's make a copy of our initial dataframe just in case anything goes wrong

morristown_venues = nearby_venues2.copy()
morristown_venues.head()


,name,id,categories,lat,lng
0,The Committed Pig,54bc2c76498e852a0330576d,Burger Joint,40.796839,-74.481776
2,HOPS,4d3b8b99a2e4b1f7a7f0e025,Bar,40.796812,-74.482786
5,Laundromat,56f5dfd9498e0f359934092b,Speakeasy,40.795506,-74.480535
6,Sushi Lounge,4b32ab3af964a520661125e3,Sushi Restaurant,40.797207,-74.482969
7,Morristown Pancake House,5442a6ea498e4795b0e18925,American Restaurant,40.795865,-74.480514


### Data  Part 2
#### This where the likes arrange into another dataframe. 

In [15]:
# add in the list of likes

morristown_venues['total likes'] = like_list
morristown_venues.head()


,name,id,categories,lat,lng,total likes
0,The Committed Pig,54bc2c76498e852a0330576d,Burger Joint,40.796839,-74.481776,133
2,HOPS,4d3b8b99a2e4b1f7a7f0e025,Bar,40.796812,-74.482786,54
5,Laundromat,56f5dfd9498e0f359934092b,Speakeasy,40.795506,-74.480535,39
6,Sushi Lounge,4b32ab3af964a520661125e3,Sushi Restaurant,40.797207,-74.482969,58
7,Morristown Pancake House,5442a6ea498e4795b0e18925,American Restaurant,40.795865,-74.480514,39


In [16]:
# now let's bin total likes

print(morristown_venues['total likes'].max())
print(morristown_venues['total likes'].min())
print(morristown_venues['total likes'].median())
print(morristown_venues['total likes'].mean())


133
0
28.0
37.7


In [17]:

# what are the bins we want to use?

print(np.percentile(morristown_venues['total likes'], 25))
print(np.percentile(morristown_venues['total likes'], 50))
print(np.percentile(morristown_venues['total likes'], 75))


17.5
28.0
48.75


In [18]:

# now we have our bin values so let's set them to the appropriate values
# less than 24, 24-45, 45-76, 76>
# poor, below avg, abv avg, great

poor = morristown_venues['total likes']<=24
below_avg = morristown_venues[(morristown_venues['total likes']>24) & (morristown_venues['total likes']<=45)]
abv_avg = morristown_venues[(morristown_venues['total likes']>45) & (morristown_venues['total likes']<=76)]
great = morristown_venues['total likes']>76


In [19]:
# let's set up a function that will re-categorize our restaurants based on likes

def conditions(s):
    if s['total likes']<=24:
        return 'poor'
    if s['total likes']<=45:
        return 'below avg'
    if s['total likes']<=76:
        return 'avg avg'
    if s['total likes']>76:
        return 'great'

morristown_venues['total likes_cat']=morristown_venues.apply(conditions, axis=1)


In [20]:
morristown_venues


,name,id,categories,lat,lng,total likes,total likes_cat
0,The Committed Pig,54bc2c76498e852a0330576d,Burger Joint,40.796839,-74.481776,133,great
2,HOPS,4d3b8b99a2e4b1f7a7f0e025,Bar,40.796812,-74.482786,54,avg avg
5,Laundromat,56f5dfd9498e0f359934092b,Speakeasy,40.795506,-74.480535,39,below avg
6,Sushi Lounge,4b32ab3af964a520661125e3,Sushi Restaurant,40.797207,-74.482969,58,avg avg
7,Morristown Pancake House,5442a6ea498e4795b0e18925,American Restaurant,40.795865,-74.480514,39,below avg
8,The Artist Baker,4b71d8c7f964a520cc5f2de3,Café,40.799101,-74.481801,19,poor
9,Millies Old World Meatballs And Pizza,50e8922ee4b097a7b77a7224,Pizza Place,40.794923,-74.479071,85,great
11,Asahi Sushi,4b7c7b55f964a5201a962fe3,Sushi Restaurant,40.797462,-74.477700,47,avg avg
13,Masago,516f0e7ce4b0ee5002c9ed70,Sushi Restaurant,40.797772,-74.483184,24,poor
15,"Revolution, A Social Brew House",57fbfd19498e5fcfe65ba2b3,Beer Bar,40.796002,-74.480858,37,below avg


In [21]:
# let's star the process for re-categorizing the categories

morristown_venues['categories'].unique()


array(['Burger Joint', 'Bar', 'Speakeasy', 'Sushi Restaurant',
       'American Restaurant', 'Café', 'Pizza Place', 'Beer Bar',
       'Indian Restaurant', 'Steakhouse', 'Frozen Yogurt Shop',
       'Mexican Restaurant', 'Empanada Restaurant', 'Coffee Shop',
       'Thai Restaurant', 'Ice Cream Shop', 'Mediterranean Restaurant',
       'New American Restaurant', 'Persian Restaurant', 'Bagel Shop',
       'Fried Chicken Joint', 'Italian Restaurant', 'Japanese Restaurant',
       'Modern European Restaurant'], dtype=object)

In [22]:
# let's create our new categories and create a function to apply those to our existing data


bars = ['Pub', 'Cocktail Bar','Beer Bar','Bar', 'Dive Bar', 'Sports Bar','Speakeasy']
other = ['Bagel Shop', 'Tea Room', 'Donut Shop', 'Coffee Shop', 'Bubble Tea Shop','Frozen Yogurt Shop', 'Sandwich Place', 'Boutique', 'Ice Cream Shop']
euro_asia_indian_food = ['Falafel Restaurant', 'Korean Restaurant','Modern European Restaurant','Sushi Restaurant', 'Mediterranean Restaurant', 'Indian Restaurant', 'Japanese Restaurant', 'Poke Place', 'Thai Restaurant', 'Vietnamese Restaurant']
mex_southam_food = ['Cuban Restaurant','Empanada Restaurant','Mexican Restaurant', 'South American Restaurant', 'Latin American Restaurant']
american_food = ['Burger Joint', 'Restaurant','Steakhouse','Fried Chicken Joint','New American Restaurant', 'American Restaurant']
italian_food = ['Italian Restaurant', 'Seafood Restaurant', 'Pizza Place']

def conditions2(s):
    if s['categories'] in bars:
        return 'bars'
    if s['categories'] in other:
        return 'other'
    if s['categories'] in euro_asia_indian_food:
        return 'euro asia indian food'
    if s['categories'] in mex_southam_food:
        return 'mex southam food'
    if s['categories'] in american_food:
        return 'american food'
    if s['categories'] in italian_food:
        return 'italian food'

morristown_venues['categories_new']=morristown_venues.apply(conditions2, axis=1)



### Data -- Part 3
#### Creating variables of  total likes and categories to cluster



In [23]:
morristown_venues


,name,id,categories,lat,lng,total likes,total likes_cat,categories_new
0,The Committed Pig,54bc2c76498e852a0330576d,Burger Joint,40.796839,-74.481776,133,great,american food
2,HOPS,4d3b8b99a2e4b1f7a7f0e025,Bar,40.796812,-74.482786,54,avg avg,bars
5,Laundromat,56f5dfd9498e0f359934092b,Speakeasy,40.795506,-74.480535,39,below avg,bars
6,Sushi Lounge,4b32ab3af964a520661125e3,Sushi Restaurant,40.797207,-74.482969,58,avg avg,euro asia indian food
7,Morristown Pancake House,5442a6ea498e4795b0e18925,American Restaurant,40.795865,-74.480514,39,below avg,american food
8,The Artist Baker,4b71d8c7f964a520cc5f2de3,Café,40.799101,-74.481801,19,poor,None
9,Millies Old World Meatballs And Pizza,50e8922ee4b097a7b77a7224,Pizza Place,40.794923,-74.479071,85,great,italian food
11,Asahi Sushi,4b7c7b55f964a5201a962fe3,Sushi Restaurant,40.797462,-74.477700,47,avg avg,euro asia indian food
13,Masago,516f0e7ce4b0ee5002c9ed70,Sushi Restaurant,40.797772,-74.483184,24,poor,euro asia indian food
15,"Revolution, A Social Brew House",57fbfd19498e5fcfe65ba2b3,Beer Bar,40.796002,-74.480858,37,below avg,bars


### Clustering Part 1
#### k-means clustering to label data

In [24]:
# one hot encoding
morristown_onehot = pd.get_dummies(morristown_venues[['categories_new', 'total likes_cat']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
morristown_onehot['Name'] = morristown_venues['name'] 

# move neighborhood column to the first column
fixed_columns = [morristown_onehot.columns[-1]] + list(morristown_onehot.columns[:-1])
morristown_onehot = morristown_onehot[fixed_columns]

morristown_onehot.head()


,Name,american food,bars,euro asia indian food,italian food,mex southam food,other,avg avg,below avg,great,poor
0,The Committed Pig,1,0,0,0,0,0,0,0,1,0
2,HOPS,0,1,0,0,0,0,1,0,0,0
5,Laundromat,0,1,0,0,0,0,0,1,0,0
6,Sushi Lounge,0,0,1,0,0,0,1,0,0,0
7,Morristown Pancake House,1,0,0,0,0,0,0,1,0,0



### Clustering Part 2
#### Labeling cluster labels back into dataframe.

In [25]:

cluster_df = morristown_onehot.drop('Name', axis=1)

k_clusters = 4

# run k-means clustering
kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(cluster_df)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]


array([1, 3, 2, 3, 2, 0, 1, 3, 0, 2], dtype=int32)

### Clustering Part 3
#### Morristown clusters.

In [26]:
morristown_venues['label'] = kmeans.labels_
morristown_venues.head()


,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
0,The Committed Pig,54bc2c76498e852a0330576d,Burger Joint,40.796839,-74.481776,133,great,american food,1
2,HOPS,4d3b8b99a2e4b1f7a7f0e025,Bar,40.796812,-74.482786,54,avg avg,bars,3
5,Laundromat,56f5dfd9498e0f359934092b,Speakeasy,40.795506,-74.480535,39,below avg,bars,2
6,Sushi Lounge,4b32ab3af964a520661125e3,Sushi Restaurant,40.797207,-74.482969,58,avg avg,euro asia indian food,3
7,Morristown Pancake House,5442a6ea498e4795b0e18925,American Restaurant,40.795865,-74.480514,39,below avg,american food,2


In [27]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=16)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i+x+(i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(morristown_venues['lat'], morristown_venues['lng'], morristown_venues['name'], morristown_venues['label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


## Clustering Part 4


### Cluster 1
#### Label --- Poor

In [28]:
morristown_venues.loc[morristown_venues['label']==0]


,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
8,The Artist Baker,4b71d8c7f964a520cc5f2de3,Café,40.799101,-74.481801,19,poor,None,0
13,Masago,516f0e7ce4b0ee5002c9ed70,Sushi Restaurant,40.797772,-74.483184,24,poor,euro asia indian food,0
17,Nagano,4b69fdb8f964a52034c02be3,Sushi Restaurant,40.797400,-74.482522,18,poor,euro asia indian food,0
26,Taro Frozen Yogurt,4ed7fa938b81b2bf292fc3c3,Frozen Yogurt Shop,40.795191,-74.479846,18,poor,other,0
35,Origin Thai Restaurant,4b7ad359f964a520123f2fe3,Thai Restaurant,40.796089,-74.480786,21,poor,euro asia indian food,0
36,Beenies Ice Cream,5928bce5d41bb74f6ec42252,Ice Cream Shop,40.797431,-74.478406,7,poor,other,0
37,SM23,4a99f0fdf964a520223120e3,Bar,40.798684,-74.480135,16,poor,bars,0
38,Pure Pita,563d26d1cd106e7049a26c7d,Mediterranean Restaurant,40.795383,-74.480475,7,poor,euro asia indian food,0
44,Morristown Bagels And Deli,51a0bb2a498e5f40f8a924de,Bagel Shop,40.796801,-74.475561,24,poor,other,0
47,Chef Fredy's Table,4a99a976f964a520ae2f20e3,American Restaurant,40.800496,-74.481724,18,poor,american food,0



### Cluster 2
#### Label --  Great

In [29]:
morristown_venues.loc[morristown_venues['label']==1]


,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
0,The Committed Pig,54bc2c76498e852a0330576d,Burger Joint,40.796839,-74.481776,133,great,american food,1
9,Millies Old World Meatballs And Pizza,50e8922ee4b097a7b77a7224,Pizza Place,40.794923,-74.479071,85,great,italian food,1
21,The Office Tavern Grill,4ade3b40f964a5201e7421e3,American Restaurant,40.795993,-74.481228,103,great,american food,1
34,Starbucks,4e1b9793c65be9fffd74c4cb,Coffee Shop,40.797400,-74.481540,109,great,other,1
41,Jockey Hollow Bar & Kitchen,5328b1c911d2e167c91df448,New American Restaurant,40.793966,-74.477363,101,great,american food,1



### Cluster 3
#### Label ---  Average 

In [30]:
morristown_venues.loc[morristown_venues['label']==2]


,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
5,Laundromat,56f5dfd9498e0f359934092b,Speakeasy,40.795506,-74.480535,39,below avg,bars,2
7,Morristown Pancake House,5442a6ea498e4795b0e18925,American Restaurant,40.795865,-74.480514,39,below avg,american food,2
15,"Revolution, A Social Brew House",57fbfd19498e5fcfe65ba2b3,Beer Bar,40.796002,-74.480858,37,below avg,bars,2
18,Mehndi,4b92a71df964a520f70d34e3,Indian Restaurant,40.798717,-74.480005,30,below avg,euro asia indian food,2
30,Tito's Burritos & Wings,4b4e1922f964a52056e126e3,Mexican Restaurant,40.797218,-74.482878,43,below avg,mex southam food,2
32,Raul's Empanadas Town,4aaad8bcf964a520b15720e3,Empanada Restaurant,40.797428,-74.477860,42,below avg,mex southam food,2
42,Marjan Fine Persian Grill,4dcb1c7345dd3831227dfe2d,Persian Restaurant,40.800642,-74.481664,31,below avg,None,2
43,SmartWorld Coffee,4b7567b6f964a520a30a2ee3,Coffee Shop,40.794608,-74.478991,37,below avg,other,2
49,Cluck U Chicken,4b57ea3df964a520184628e3,Fried Chicken Joint,40.797878,-74.476529,44,below avg,american food,2
50,The Godfather Pizzeria,52cb5dbd498e7d6760b5b6b8,Italian Restaurant,40.797302,-74.476090,26,below avg,italian food,2


### Cluster 4 
#### Label --- Great


In [31]:
morristown_venues.loc[morristown_venues['label']==3]

,name,id,categories,lat,lng,total likes,total likes_cat,categories_new,label
2,HOPS,4d3b8b99a2e4b1f7a7f0e025,Bar,40.796812,-74.482786,54,avg avg,bars,3
6,Sushi Lounge,4b32ab3af964a520661125e3,Sushi Restaurant,40.797207,-74.482969,58,avg avg,euro asia indian food,3
11,Asahi Sushi,4b7c7b55f964a5201a962fe3,Sushi Restaurant,40.797462,-74.477700,47,avg avg,euro asia indian food,3
23,south+pine american eatery,5570eb60498e84218b6ae325,American Restaurant,40.794382,-74.478447,71,avg avg,american food,3
25,Roots Steakhouse,4d51fd29e0275481c069b0b6,Steakhouse,40.796435,-74.481269,54,avg avg,american food,3
40,Macho Nacho,4ad8ae12f964a520aa1321e3,Mexican Restaurant,40.797932,-74.476711,70,avg avg,mex southam food,3


## Conclusion

####  In conclusion,  Morristown offers a lot venues for a town of its size.  Most of these places are located in the downtown area.  It has substatial amount of venues that are like from average to great.  Most of the best rank restaurants were from the American food category.  Most of the worst restaurants were from the Euro Asia Indian food category.  From the data, one can determine that most restaurants have very poor rating according to the number of likes. The bars were evenly spread out when it came to ratings. None of the bars had great rating.  There is still room for improvement when it comes to the overall rating of the venues.
